In [1]:
import os
import yaml
import json
import pandas as pd

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import wombat.bsub as bsub

load sample data

In [5]:
m = json.load(open('../tests/data/tinjasmine/C3L-01232.json'))
m

{'case_id': 'C3L-01232',
 'disease': 'GBM',
 'WXS': {'normal': {'id': 'C3L-01232.WXS.N.hg38',
   'barcode': 'CPT0240430002',
   'filepath': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/a65f1db8-170a-4ec2-a20b-e6deff964e2c/54352f31-de32-4a52-b0c0-1ab882496fc3_wxs_gdc_realn.bam'},
  'tumor': {'id': 'C3L-01232.WXS.T.hg38',
   'barcode': 'CPT0239440006',
   'filepath': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/6da28b7b-b148-4c49-97ee-d4587b058a46/218349a8-16e2-4bcd-9c9a-690bc5152441_wxs_gdc_realn.bam'}},
 'WGS': {'normal': {'id': 'C3L-01232.WGS.N.hg38',
   'filepath': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/c7777e0c-2447-4d47-aa47-e65bed9a8e47/54352f31-de32-4a52-b0c0-1ab882496fc3_wgs_gdc_realn.bam'},
  'tumor': {'id': 'C3L-01232.WGS.T.hg38',
   'filepath': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/7794e4ec-ebc2-40ca-9e51-

make test input yaml

In [6]:
template = yaml.safe_load(open('/home/estorrs/pecgs-pipeline/cwl/msisensor/template.msisensor.yaml'))
template

{'microsatellite': {'class': 'File', 'path': 'a/file/path'},
 'minimal_homopolymer_size': 'a_int',
 'minimal_microsatellite_size': 'a_int',
 'normal_bam': {'class': 'File', 'path': 'a/file/path'},
 'threads': 'a_int',
 'tumor_bam': {'class': 'File', 'path': 'a/file/path'}}

In [7]:
template['microsatellite']['path'] = '/storage1/fs1/dinglab/Active/Projects/PECGS/ref_genome/hg38.microsatellite'
template['minimal_homopolymer_size'] = 1
template['minimal_microsatellite_size'] = 1
template['threads'] = 1
template['normal_bam']['path'] = m['WXS']['normal']['filepath']
template['tumor_bam']['path'] = m['WXS']['tumor']['filepath']

In [8]:
template

{'microsatellite': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/PECGS/ref_genome/hg38.microsatellite'},
 'minimal_homopolymer_size': 1,
 'minimal_microsatellite_size': 1,
 'normal_bam': {'class': 'File',
  'path': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/a65f1db8-170a-4ec2-a20b-e6deff964e2c/54352f31-de32-4a52-b0c0-1ab882496fc3_wxs_gdc_realn.bam'},
 'threads': 1,
 'tumor_bam': {'class': 'File',
  'path': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/6da28b7b-b148-4c49-97ee-d4587b058a46/218349a8-16e2-4bcd-9c9a-690bc5152441_wxs_gdc_realn.bam'}}

In [10]:
yaml.safe_dump(template, open('../tests/data/msisensor/C3L-01232_inputs.yaml', 'w'))

make cromwell compute1 config

In [11]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/msisensor/C3L-01232'
output_fp = '../tests/data/msisensor/C3L-01232.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

make execution script

In [12]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/msisensor'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'C3L-01232.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'cwl', 'msisensor', 'msisensor.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'C3L-01232_inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab/Active', '/storage1/fs1/m.wyczalkowski/Active']

In [13]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [14]:
start_commands, run_command = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes)
filepath = os.path.join('../tests/data/msisensor/start_server.sh')
bsub.write_command_file(start_commands, filepath)
filepath = os.path.join('../tests/data/msisensor/run_job.sh')
bsub.write_command_file([run_command], filepath)

In [15]:
for c in start_commands: print(c)

source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/msisensor:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/msisensor /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/msisensor/C3L-01232:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/msisensor/C3L-01232 /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active"
bgadd -L 10 /estorrs/test_cromwell
bsub -q dinglab-interactive -G compute-dinglab -a 'docker(mwyczalkowski/cromwell-runner)' -Is '/bin/bash'


In [16]:
print(run_command)

/usr/bin/java -Dconfig.file=/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/msisensor/C3L-01232.cromwell-config-db.compute1.dat -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore -jar /usr/local/cromwell/cromwell-47.jar run -t cwl -i /storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/msisensor/C3L-01232_inputs.yaml /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/cwl/msisensor/msisensor.cwl
